<a href="https://colab.research.google.com/github/KorStats/classification-project/blob/main/%EC%A6%9D%EB%B6%84/(%EB%85%B8%EC%9D%B4)_%EC%9B%90%EB%B3%B8G%EB%8D%B0%EC%9D%B4%ED%84%B0_%EC%A6%9D%EB%B6%84.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

## 구글드라이브에서 파일 읽어오기

In [1]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [2]:
from google.colab import drive
drive.mount('/content/gdrive')
!ln -s /content/gdrive/My\ Drive/ /mydrive
!ls /mydrive

Mounted at /content/gdrive
 기본   산업분류자동화	'Colab Notebooks'  'My Drive'


In [3]:
import pandas as pd
df=pd.read_csv('/mydrive/산업분류자동화/dataset/spell_check/G_spell_check.csv', sep = ",", encoding = "UTF-8")

## 데이터 전처리

In [4]:
df=df.reset_index(drop=True)
df

,AI_id,digit_1,digit_2,digit_3,text
0,id_0000013,G,47,474,매장에서 일반인 대상으로 의류소매
1,id_0000060,G,47,474,매장에서 일반인대상으로 의류소매
2,id_0000086,G,47,474,사업장에서 일반인대상 의류판매
3,id_0000109,G,47,474,일반고객에게 신발매장에서 일반인을 대상으로 신발
4,id_0000112,G,47,474,일반소비자 가정용 직물제품소매점 커튼 방석 침구류등 소매
...,...,...,...,...,...
246467,id_0994580,G,45,453,사업장 일반 오토바이
246468,id_0995378,G,45,453,일반 소비자 가게에서 오토바이 소매
246469,id_0995841,G,45,453,사륜 산악자전거 일반인 대상으로 수입 판매
246470,id_0998347,G,45,453,사무실에서 수입하여 전기오토바이도 매


In [5]:
df['digit_3'].value_counts().sort_index()

451     2268
452     4696
453      471
461     2328
462     2585
463    17873
464    18453
465    14652
466     8154
467    12096
468      694
471    26189
472    25668
473     9312
474    36453
475    13031
476     7188
477     4577
478    30566
479     9218
Name: digit_3, dtype: int64

## **# 데이터 증분**

In [6]:
# 데이터 증분 코드
import random
import pickle
import re
def get_only_hangul(line):
	parseText= re.compile('/ ^[ㄱ-ㅎㅏ-ㅣ가-힣]*$/').sub('',line)
	return parseText

def synonym_replacement(words, n):
	new_words = words.copy()
	random_word_list = list(set([word for word in words]))
	random.shuffle(random_word_list)
	num_replaced = 0
	for random_word in random_word_list:
		synonyms = get_synonyms(random_word)
		if len(synonyms) >= 1:
			synonym = random.choice(list(synonyms))
			new_words = [synonym if word == random_word else word for word in new_words]
			num_replaced += 1
		if num_replaced >= n:
			break
	if len(new_words) != 0:
		sentence = ' '.join(new_words)
		new_words = sentence.split(" ")
	else:
		new_words = ""
	return new_words

def get_synonyms(word):
	synomyms = []
	try:
		for syn in wordnet[word]:
			for s in syn:
				synomyms.append(s)
	except:
		pass
	return synomyms

def random_deletion(words, p):
	if len(words) == 1:
		return words

	new_words = []
	for word in words:
		r = random.uniform(0, 1)
		if r > p:
			new_words.append(word)
	if len(new_words) == 0:
		rand_int = random.randint(0, len(words)-1)
		return [words[rand_int]]
	return new_words

########################################################################
# Random swap
# Randomly swap two words in the sentence n times
########################################################################
def random_swap(words, n):
	new_words = words.copy()
	for _ in range(n):
		new_words = swap_word(new_words)
	return new_words

def swap_word(new_words):
	random_idx_1 = random.randint(0, len(new_words)-1)
	random_idx_2 = random_idx_1
	counter = 0
	while random_idx_2 == random_idx_1:
		random_idx_2 = random.randint(0, len(new_words)-1)
		counter += 1
		if counter > 3:
			return new_words
	new_words[random_idx_1], new_words[random_idx_2] = new_words[random_idx_2], new_words[random_idx_1]
	return new_words

########################################################################
# Random insertion
# Randomly insert n words into the sentence
########################################################################
def random_insertion(words, n):
	new_words = words.copy()
	for _ in range(n):
		add_word(new_words)	
	return new_words

def add_word(new_words):
	synonyms = []
	counter = 0
	while len(synonyms) < 1:
		if len(new_words) >= 1:
			random_word = new_words[random.randint(0, len(new_words)-1)]
			synonyms = get_synonyms(random_word)
			counter += 1
		else:
			random_word = ""
		if counter >= 10:
			return
		
	random_synonym = synonyms[0]
	random_idx = random.randint(0, len(new_words)-1)
	new_words.insert(random_idx, random_synonym)

def EDA(sentence, alpha_sr=0.1, alpha_ri=0.1, alpha_rs=0.1, p_rd=0.1, num_aug=9):
	sentence = get_only_hangul(sentence)
	words = sentence.split(' ')
	words = [word for word in words if word is not ""]
	num_words = len(words)
	augmented_sentences = []
	num_new_per_technique = int(num_aug/4) + 1
	n_sr = max(1, int(alpha_sr*num_words))
	n_ri = max(1, int(alpha_ri*num_words))
	n_rs = max(1, int(alpha_rs*num_words))
	# sr
	for _ in range(num_new_per_technique):
		a_words = synonym_replacement(words, n_sr)
		augmented_sentences.append(' '.join(a_words))
	# ri
	for _ in range(num_new_per_technique):
		a_words = random_insertion(words, n_ri)
		augmented_sentences.append(' '.join(a_words))
	# rs
	for _ in range(num_new_per_technique):
		a_words = random_swap(words, n_rs)
		augmented_sentences.append(" ".join(a_words))
	# rd
	for _ in range(num_new_per_technique):
		a_words = random_deletion(words, p_rd)
		augmented_sentences.append(" ".join(a_words))
	augmented_sentences = [get_only_hangul(sentence) for sentence in augmented_sentences]
	random.shuffle(augmented_sentences)
	if num_aug >= 1:
		augmented_sentences = augmented_sentences[:num_aug]
	else:
		keep_prob = num_aug / len(augmented_sentences)
		augmented_sentences = [s for s in augmented_sentences if random.uniform(0, 1) < keep_prob]
	augmented_sentences.append(sentence)
	return augmented_sentences

In [7]:
df0=df.loc[(df['digit_3'] == 451)]  #*10
df1=df.loc[(df['digit_3'] == 452)]  #*5
df2=df.loc[(df['digit_3'] == 453)]  #*50
df3=df.loc[(df['digit_3'] == 461)]   #*10
df4=df.loc[(df['digit_3'] == 462)]   #*10
df5=df.loc[(df['digit_3'] == 463)]   #*2
df6=df.loc[(df['digit_3'] == 464)]   #*2
df7=df.loc[(df['digit_3'] == 465)]   #*2
df8=df.loc[(df['digit_3'] == 466)]   #*3
df9=df.loc[(df['digit_3'] == 467)]   #*2
df10=df.loc[(df['digit_3'] == 468)]  #*50
df11=df.loc[(df['digit_3'] == 471)]   
df12=df.loc[(df['digit_3'] == 472)]   
df13=df.loc[(df['digit_3'] == 473)]   #*3
df14=df.loc[(df['digit_3'] == 474)]   
df15=df.loc[(df['digit_3'] == 475)]   #*2
df16=df.loc[(df['digit_3'] == 476)]   #*3
df17=df.loc[(df['digit_3'] == 477)]   #*5
df18=df.loc[(df['digit_3'] == 478)]   
df19=df.loc[(df['digit_3'] == 479)]   #*3

In [8]:
df1=df1.reset_index(drop=True)
df2=df2.reset_index(drop=True)
df3=df3.reset_index(drop=True)
df4=df4.reset_index(drop=True)
df5=df5.reset_index(drop=True)
df6=df6.reset_index(drop=True)
df7=df7.reset_index(drop=True)
df8=df8.reset_index(drop=True)
df9=df9.reset_index(drop=True)
df10=df10.reset_index(drop=True)
df11=df11.reset_index(drop=True)
df12=df12.reset_index(drop=True)
df13=df13.reset_index(drop=True)
df14=df14.reset_index(drop=True)
df15=df15.reset_index(drop=True)
df16=df16.reset_index(drop=True)
df17=df17.reset_index(drop=True)
df18=df18.reset_index(drop=True)
df19=df19.reset_index(drop=True)

In [9]:
df_1=pd.concat([df11, df12, df14, df18]) # *1
df_2=pd.concat([df5, df6, df7, df9, df15]) # *2
df_3=pd.concat([df8, df13, df16, df19]) # *3
df_5=pd.concat([df1, df17]) # *5
df_10=pd.concat([df0, df3, df4]) # *10 묶음
df_50=pd.concat([df2, df10])

In [10]:
df_1=df_1.reset_index(drop=True)
df_2=df_2.reset_index(drop=True)
df_3=df_3.reset_index(drop=True)
df_5=df_5.reset_index(drop=True)
df_10=df_10.reset_index(drop=True)
df_50=df_50.reset_index(drop=True)

In [11]:
df1.head()

,AI_id,digit_1,digit_2,digit_3,text
0,id_0000085,G,45,452,영업장에서 고객 대상으로 자동차 부품 판매
1,id_0000517,G,45,452,자동차 내장재 산업 사용자 대상으로 pvc 인조가죽 시트 등 자재
2,id_0001076,G,45,452,자동차 내 장품 가게에서 일반인을 대상으로 소매 자동차 오디오 설치
3,id_0002104,G,45,452,사무실에서 일반인 대상 내비게이션 판매
4,id_0002295,G,45,452,쇼핑몰에서 기업회원 전용 타이어 판매


In [12]:
temp_result=[]
n=0
id_list=[]
digit_1_list=[]
digit_2_list=[]
digit_3_list=[]

temp = {'AI_id': [], 'digit_1': [], 'digit_2' : [], 'digit_3' : [], 'text' : []}
temp = pd.DataFrame(temp)

for sentence in df_2['text']:
  result_list=EDA(sentence, alpha_sr=0.0, alpha_ri=0.0, alpha_rs=0.5, p_rd=0.5, num_aug=1)
  id_list=[str(df_2['AI_id'][n]), 'p'+str(df_2['AI_id'][n])]
  digit_1_list=[df_2['digit_1'][n], df_2['digit_1'][n]]
  digit_2_list=[df_2['digit_2'][n], df_2['digit_2'][n]]
  digit_3_list=[df_2['digit_3'][n], df_2['digit_3'][n]]
  data_to_insert={'AI_id' : id_list, 'digit_1' : digit_1_list, 'digit_2' : digit_2_list, 'digit_3': digit_3_list, 'text' : result_list}
  df_2_to_insert = pd.DataFrame(data_to_insert)
  temp= temp.append(df_2_to_insert, ignore_index=True)
  n=n+1

df_2=temp

In [13]:
temp_result=[]
n=0
id_list=[]
digit_1_list=[]
digit_2_list=[]
digit_3_list=[]

temp = {'AI_id': [], 'digit_1': [], 'digit_2' : [], 'digit_3' : [], 'text' : []}
temp = pd.DataFrame(temp)

for sentence in df_3['text']:
  result_list=EDA(sentence, alpha_sr=0.0, alpha_ri=0.0, alpha_rs=0.5, p_rd=0.5, num_aug=2)
  id_list=[str(df_3['AI_id'][n]), 'p'+str(df_3['AI_id'][n]), 'p'+str(df_3['AI_id'][n])]
  digit_1_list=[df_3['digit_1'][n], df_3['digit_1'][n], df_3['digit_1'][n]]
  digit_2_list=[df_3['digit_2'][n], df_3['digit_2'][n], df_3['digit_2'][n]]
  digit_3_list=[df_3['digit_3'][n], df_3['digit_3'][n], df_3['digit_3'][n]]
  data_to_insert={'AI_id' : id_list, 'digit_1' : digit_1_list, 'digit_2' : digit_2_list, 'digit_3': digit_3_list, 'text' : result_list}
  df_3_to_insert = pd.DataFrame(data_to_insert)
  temp= temp.append(df_3_to_insert, ignore_index=True)
  n=n+1

df_3=temp

In [14]:
temp_result=[]
n=0
id_list=[]
digit_1_list=[]
digit_2_list=[]
digit_3_list=[]

temp = {'AI_id': [], 'digit_1': [], 'digit_2' : [], 'digit_3' : [], 'text' : []}
temp = pd.DataFrame(temp)

for sentence in df_5['text']:
  result_list=EDA(sentence, alpha_sr=0.0, alpha_ri=0.0, alpha_rs=0.5, p_rd=0.5, num_aug=4)
  id_list=[str(df_5['AI_id'][n]), 'p'+str(df_5['AI_id'][n]), 'p'+str(df_5['AI_id'][n]), 'p'+str(df_5['AI_id'][n]), 'p'+str(df_5['AI_id'][n])]
  digit_1_list=[df_5['digit_1'][n], df_5['digit_1'][n], df_5['digit_1'][n], df_5['digit_1'][n], df_5['digit_1'][n]]
  digit_2_list=[df_5['digit_2'][n], df_5['digit_2'][n], df_5['digit_2'][n], df_5['digit_2'][n], df_5['digit_2'][n]]
  digit_3_list=[df_5['digit_3'][n], df_5['digit_3'][n], df_5['digit_3'][n], df_5['digit_3'][n], df_5['digit_3'][n]]
  data_to_insert={'AI_id' : id_list, 'digit_1' : digit_1_list, 'digit_2' : digit_2_list, 'digit_3': digit_3_list, 'text' : result_list}
  df_5_to_insert = pd.DataFrame(data_to_insert)
  temp= temp.append(df_5_to_insert, ignore_index=True)
  n=n+1

df_5=temp

In [16]:
temp_result=[]
n=0
id_list=[]
digit_1_list=[]
digit_2_list=[]
digit_3_list=[]

temp = {'AI_id': [], 'digit_1': [], 'digit_2' : [], 'digit_3' : [], 'text' : []}
temp = pd.DataFrame(temp)

for sentence in df_10['text']:
  result_list=EDA(sentence, alpha_sr=0.0, alpha_ri=0.0, alpha_rs=0.5, p_rd=0.5, num_aug=9)
  id_list=[str(df_10['AI_id'][n]), 'p'+str(df_10['AI_id'][n]), 'p'+str(df_10['AI_id'][n]), 'p'+str(df_10['AI_id'][n]), 'p'+str(df_10['AI_id'][n]), 'p'+str(df_10['AI_id'][n]), 'p'+str(df_10['AI_id'][n]), 'p'+str(df_10['AI_id'][n]), 'p'+str(df_10['AI_id'][n]), 'p'+str(df_10['AI_id'][n])]
  digit_1_list=[df_10['digit_1'][n], df_10['digit_1'][n], df_10['digit_1'][n], df_10['digit_1'][n], df_10['digit_1'][n], df_10['digit_1'][n], df_10['digit_1'][n], df_10['digit_1'][n], df_10['digit_1'][n], df_10['digit_1'][n]]
  digit_2_list=[df_10['digit_2'][n], df_10['digit_2'][n], df_10['digit_2'][n], df_10['digit_2'][n], df_10['digit_2'][n], df_10['digit_2'][n], df_10['digit_2'][n], df_10['digit_2'][n], df_10['digit_2'][n], df_10['digit_2'][n]]
  digit_3_list=[df_10['digit_3'][n], df_10['digit_3'][n], df_10['digit_3'][n], df_10['digit_3'][n], df_10['digit_3'][n], df_10['digit_3'][n], df_10['digit_3'][n], df_10['digit_3'][n], df_10['digit_3'][n], df_10['digit_3'][n]]
  data_to_insert={'AI_id' : id_list, 'digit_1' : digit_1_list, 'digit_2' : digit_2_list, 'digit_3': digit_3_list, 'text' : result_list}
  df_10_to_insert = pd.DataFrame(data_to_insert)
  temp= temp.append(df_10_to_insert, ignore_index=True)
  n=n+1

df_10=temp

In [17]:
temp_result=[]
n=0
id_list=[]
digit_1_list=[]
digit_2_list=[]
digit_3_list=[]

temp = {'AI_id': [], 'digit_1': [], 'digit_2' : [], 'digit_3' : [], 'text' : []}
temp = pd.DataFrame(temp)

for sentence in df_50['text']:
  result_list=EDA(sentence, alpha_sr=0.0, alpha_ri=0.0, alpha_rs=0.5, p_rd=0.5, num_aug=49)
  id_list=[str(df_50['AI_id'][n]), 'p'+str(df_50['AI_id'][n]), 'p'+str(df_50['AI_id'][n]), 'p'+str(df_50['AI_id'][n]), 'p'+str(df_50['AI_id'][n]), 'p'+str(df_50['AI_id'][n]), 'p'+str(df_50['AI_id'][n]), 'p'+str(df_50['AI_id'][n]), 'p'+str(df_50['AI_id'][n]), 'p'+str(df_50['AI_id'][n]), 'p'+str(df_50['AI_id'][n]), 'p'+str(df_50['AI_id'][n]), 'p'+str(df_50['AI_id'][n]), 'p'+str(df_50['AI_id'][n]), 'p'+str(df_50['AI_id'][n]), 'p'+str(df_50['AI_id'][n]), 'p'+str(df_50['AI_id'][n]), 'p'+str(df_50['AI_id'][n]), 'p'+str(df_50['AI_id'][n]), 'p'+str(df_50['AI_id'][n]), 'p'+str(df_50['AI_id'][n]), 'p'+str(df_50['AI_id'][n]), 'p'+str(df_50['AI_id'][n]), 'p'+str(df_50['AI_id'][n]), 'p'+str(df_50['AI_id'][n]), 'p'+str(df_50['AI_id'][n]), 'p'+str(df_50['AI_id'][n]), 'p'+str(df_50['AI_id'][n]), 'p'+str(df_50['AI_id'][n]), 'p'+str(df_50['AI_id'][n]), 'p'+str(df_50['AI_id'][n]), 'p'+str(df_50['AI_id'][n]), 'p'+str(df_50['AI_id'][n]), 'p'+str(df_50['AI_id'][n]), 'p'+str(df_50['AI_id'][n]), 'p'+str(df_50['AI_id'][n]), 'p'+str(df_50['AI_id'][n]), 'p'+str(df_50['AI_id'][n]), 'p'+str(df_50['AI_id'][n]), 'p'+str(df_50['AI_id'][n]), 'p'+str(df_50['AI_id'][n]), 'p'+str(df_50['AI_id'][n]), 'p'+str(df_50['AI_id'][n]), 'p'+str(df_50['AI_id'][n]), 'p'+str(df_50['AI_id'][n]), 'p'+str(df_50['AI_id'][n]), 'p'+str(df_50['AI_id'][n]), 'p'+str(df_50['AI_id'][n]), 'p'+str(df_50['AI_id'][n]), 'p'+str(df_50['AI_id'][n])]
  digit_1_list=[df_50['digit_1'][n], df_50['digit_1'][n], df_50['digit_1'][n], df_50['digit_1'][n], df_50['digit_1'][n], df_50['digit_1'][n], df_50['digit_1'][n], df_50['digit_1'][n], df_50['digit_1'][n], df_50['digit_1'][n], df_50['digit_1'][n], df_50['digit_1'][n], df_50['digit_1'][n], df_50['digit_1'][n], df_50['digit_1'][n], df_50['digit_1'][n], df_50['digit_1'][n], df_50['digit_1'][n], df_50['digit_1'][n], df_50['digit_1'][n], df_50['digit_1'][n], df_50['digit_1'][n], df_50['digit_1'][n], df_50['digit_1'][n], df_50['digit_1'][n], df_50['digit_1'][n], df_50['digit_1'][n], df_50['digit_1'][n], df_50['digit_1'][n], df_50['digit_1'][n], df_50['digit_1'][n], df_50['digit_1'][n], df_50['digit_1'][n], df_50['digit_1'][n], df_50['digit_1'][n], df_50['digit_1'][n], df_50['digit_1'][n], df_50['digit_1'][n], df_50['digit_1'][n], df_50['digit_1'][n], df_50['digit_1'][n], df_50['digit_1'][n], df_50['digit_1'][n], df_50['digit_1'][n], df_50['digit_1'][n], df_50['digit_1'][n], df_50['digit_1'][n], df_50['digit_1'][n], df_50['digit_1'][n], df_50['digit_1'][n]]
  digit_2_list=[df_50['digit_2'][n], df_50['digit_2'][n], df_50['digit_2'][n], df_50['digit_2'][n], df_50['digit_2'][n], df_50['digit_2'][n], df_50['digit_2'][n], df_50['digit_2'][n], df_50['digit_2'][n], df_50['digit_2'][n], df_50['digit_2'][n], df_50['digit_2'][n], df_50['digit_2'][n], df_50['digit_2'][n], df_50['digit_2'][n], df_50['digit_2'][n], df_50['digit_2'][n], df_50['digit_2'][n], df_50['digit_2'][n], df_50['digit_2'][n], df_50['digit_2'][n], df_50['digit_2'][n], df_50['digit_2'][n], df_50['digit_2'][n], df_50['digit_2'][n], df_50['digit_2'][n], df_50['digit_2'][n], df_50['digit_2'][n], df_50['digit_2'][n], df_50['digit_2'][n], df_50['digit_2'][n], df_50['digit_2'][n], df_50['digit_2'][n], df_50['digit_2'][n], df_50['digit_2'][n], df_50['digit_2'][n], df_50['digit_2'][n], df_50['digit_2'][n], df_50['digit_2'][n], df_50['digit_2'][n], df_50['digit_2'][n], df_50['digit_2'][n], df_50['digit_2'][n], df_50['digit_2'][n], df_50['digit_2'][n], df_50['digit_2'][n], df_50['digit_2'][n], df_50['digit_2'][n], df_50['digit_2'][n], df_50['digit_2'][n]]
  digit_3_list=[df_50['digit_3'][n], df_50['digit_3'][n], df_50['digit_3'][n], df_50['digit_3'][n], df_50['digit_3'][n], df_50['digit_3'][n], df_50['digit_3'][n], df_50['digit_3'][n], df_50['digit_3'][n], df_50['digit_3'][n], df_50['digit_3'][n], df_50['digit_3'][n], df_50['digit_3'][n], df_50['digit_3'][n], df_50['digit_3'][n], df_50['digit_3'][n], df_50['digit_3'][n], df_50['digit_3'][n], df_50['digit_3'][n], df_50['digit_3'][n], df_50['digit_3'][n], df_50['digit_3'][n], df_50['digit_3'][n], df_50['digit_3'][n], df_50['digit_3'][n], df_50['digit_3'][n], df_50['digit_3'][n], df_50['digit_3'][n], df_50['digit_3'][n], df_50['digit_3'][n], df_50['digit_3'][n], df_50['digit_3'][n], df_50['digit_3'][n], df_50['digit_3'][n], df_50['digit_3'][n], df_50['digit_3'][n], df_50['digit_3'][n], df_50['digit_3'][n], df_50['digit_3'][n], df_50['digit_3'][n], df_50['digit_3'][n], df_50['digit_3'][n], df_50['digit_3'][n], df_50['digit_3'][n], df_50['digit_3'][n], df_50['digit_3'][n], df_50['digit_3'][n], df_50['digit_3'][n], df_50['digit_3'][n], df_50['digit_3'][n]]
  data_to_insert={'AI_id' : id_list, 'digit_1' : digit_1_list, 'digit_2' : digit_2_list, 'digit_3': digit_3_list, 'text' : result_list}
  df_50_to_insert = pd.DataFrame(data_to_insert)
  temp= temp.append(df_50_to_insert, ignore_index=True)
  n=n+1

df_50=temp

기존 데이터 합치기

In [18]:
df=pd.concat([df_1, df_2, df_3, df_5, df_10, df_50])

In [19]:
df['digit_3'].value_counts().sort_index()

451.0    22680
452.0    23480
453.0    23550
461.0    23280
462.0    25850
463.0    35746
464.0    36906
465.0    29304
466.0    24462
467.0    24192
468.0    34700
471.0    26189
472.0    25668
473.0    27936
474.0    36453
475.0    26062
476.0    21564
477.0    22885
478.0    30566
479.0    27654
Name: digit_3, dtype: int64

In [ ]:
df

In [20]:
df.to_csv('노이)df_G_inc_again.csv', index=False, encoding='utf-8-sig')